# Exploration
This notebook is for tuning hyperparameters and finding the most accurate architecture for this aplication.

In [1]:
from utils import *
from data.data_utils import *

In [2]:
import pandas as pd

# Data importation

dataframe = pd.read_csv('data/data.csv')

#Getting the columns
columns = dataframe.columns.values

dataframe


,Unnamed: 0,Unnamed: 1,count,hr,cosine,time,psg
0,1449548,0,52.235491,0.614925,-0.258819,0.000000,0.0
1,1449548,1,58.012114,0.608567,-0.260926,0.008333,0.0
2,1449548,2,53.536372,0.617486,-0.263031,0.016667,0.0
3,1449548,3,41.277177,0.609667,-0.265135,0.025000,0.0
4,1449548,4,27.159358,0.606250,-0.267238,0.033333,0.0
...,...,...,...,...,...,...,...
25476,9618981,938,0.000000,0.092941,-0.740218,7.816667,0.0
25477,9618981,939,0.000000,0.085237,-0.738749,7.825000,0.0
25478,9618981,940,0.000000,0.083318,-0.737277,7.833333,0.0
25479,9618981,941,0.000000,0.084314,-0.735802,7.841667,0.0


In [ ]:
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split, TensorDataset

#Defining input features and target feature

input_cols = columns[[2,3]]
targets = columns[-1]

#Getting the tensors
inputs, targets = dataframe_to_torch(dataframe,input_cols, targets)

#Creating the dataset
dataset = TensorDataset(inputs, targets)

#Cross-validation set:20% training set: 80%

val_size = round(0.2*len(dataset))

train_ds, val_ds = random_split(dataset , [len(dataset) - val_size, val_size])

batch_size = 128  # Change based in dataset size and  GPU capacity (also known as a hyperparameter)

train_loader = DataLoader(train_ds, batch_size, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_ds, batch_size*2, num_workers=4, pin_memory=True)

In [ ]:
# Input and ouput values of the hidden layers
architecture = (256, 128, 64, 32)

# Model definition
model = DeepNeuralNetwork(2, *architecture)

In [ ]:
#if meta-learning
architecture = (256, 128, 64, 32)

number_of_architectures = 3

models = []

for i in range(number_of_architectures):
    model = DeepNeuralNetwork(2, *architecture)
    models.append(model)

In [ ]:
#Model architecture
print(model)

In [ ]:
#Hyperparameters
epochs = 10
max_lr = 0.0007
grad_clip = 0.007
weight_decay = 1e-5
opt_func = torch.optim.Adam

#Training
history = []

history += fit_one_cycle(epochs, max_lr, model, train_loader, val_loader, weight_decay, grad_clip, opt_func)

In [ ]:
plot_losses(history)